# Loop 5 Analysis: Submission Failure Investigation

The submission failed with 'Overlapping trees in group 002'. Let me investigate what went wrong and find a path forward.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

getcontext().prec = 30
scale_factor = Decimal('1e15')

print('Libraries loaded')

Libraries loaded


In [2]:
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))

print('ChristmasTree class defined')

ChristmasTree class defined


In [3]:
def has_collision(trees):
    """Check for collisions between trees using Shapely."""
    if len(trees) <= 1:
        return False
    for i, tree1 in enumerate(trees):
        for j, tree2 in enumerate(trees):
            if i < j:
                if tree1.polygon.intersects(tree2.polygon) and not tree1.polygon.touches(tree2.polygon):
                    return True
    return False

def load_configuration_from_df(n, df):
    group_data = df[df['id'].str.startswith(f'{n:03d}_')]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row['x'])[1:] if str(row['x']).startswith('s') else str(row['x'])
        y = str(row['y'])[1:] if str(row['y']).startswith('s') else str(row['y'])
        deg = str(row['deg'])[1:] if str(row['deg']).startswith('s') else str(row['deg'])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

print('Helper functions defined')

Helper functions defined


In [4]:
# Load the failed submission and check for overlaps
print('Loading failed submission...')
failed_df = pd.read_csv('/home/submission/submission.csv')

# Check N=2 specifically (the error mentioned group 002)
trees_2 = load_configuration_from_df(2, failed_df)
print(f'N=2 trees: {len(trees_2)}')
for i, t in enumerate(trees_2):
    print(f'  Tree {i}: x={float(t.center_x):.6f}, y={float(t.center_y):.6f}, angle={float(t.angle):.6f}')

has_overlap_2 = has_collision(trees_2)
print(f'N=2 has overlap: {has_overlap_2}')

Loading failed submission...
N=2 trees: 2
  Tree 0: x=0.154097, y=-0.038541, angle=203.629378
  Tree 1: x=-0.154097, y=-0.561459, angle=23.629378
N=2 has overlap: True


In [5]:
# Load the repaired baseline and compare
print('\nLoading repaired baseline...')
baseline_df = pd.read_csv('/home/code/experiments/003_preoptimized/repaired_baseline.csv')

trees_2_baseline = load_configuration_from_df(2, baseline_df)
print(f'Baseline N=2 trees: {len(trees_2_baseline)}')
for i, t in enumerate(trees_2_baseline):
    print(f'  Tree {i}: x={float(t.center_x):.6f}, y={float(t.center_y):.6f}, angle={float(t.angle):.6f}')

has_overlap_2_baseline = has_collision(trees_2_baseline)
print(f'Baseline N=2 has overlap: {has_overlap_2_baseline}')


Loading repaired baseline...
Baseline N=2 trees: 2
  Tree 0: x=0.154097, y=-0.038541, angle=203.629378
  Tree 1: x=-0.154097, y=-0.561459, angle=23.629378
Baseline N=2 has overlap: False


In [6]:
# Check all N values for overlaps in both files
print('\nChecking all N values for overlaps...')

failed_overlaps = []
baseline_overlaps = []

for n in range(1, 201):
    trees_failed = load_configuration_from_df(n, failed_df)
    trees_baseline = load_configuration_from_df(n, baseline_df)
    
    if has_collision(trees_failed):
        failed_overlaps.append(n)
    if has_collision(trees_baseline):
        baseline_overlaps.append(n)

print(f'Failed submission overlaps: {len(failed_overlaps)} N values')
if failed_overlaps:
    print(f'  N values with overlaps: {failed_overlaps[:20]}...' if len(failed_overlaps) > 20 else f'  N values with overlaps: {failed_overlaps}')

print(f'\nBaseline overlaps: {len(baseline_overlaps)} N values')
if baseline_overlaps:
    print(f'  N values with overlaps: {baseline_overlaps}')


Checking all N values for overlaps...


Failed submission overlaps: 198 N values
  N values with overlaps: [2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]...

Baseline overlaps: 0 N values


In [7]:
# The issue is clear - the SA optimizer introduced overlaps even though it claimed not to
# Let me check if the repaired baseline is truly overlap-free
print('Verifying repaired baseline is overlap-free...')

for n in range(1, 201):
    trees = load_configuration_from_df(n, baseline_df)
    if has_collision(trees):
        print(f'  N={n}: OVERLAP!')

print('Verification complete.')

Verifying repaired baseline is overlap-free...


Verification complete.


In [8]:
# The repaired baseline should be overlap-free. Let me copy it directly to submission
# and verify it works
import shutil

print('Copying repaired baseline to submission...')
shutil.copy('/home/code/experiments/003_preoptimized/repaired_baseline.csv', '/home/submission/submission.csv')

# Verify the copy
verify_df = pd.read_csv('/home/submission/submission.csv')
print(f'Submission rows: {len(verify_df)}')

# Calculate score
def get_side_length(trees):
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1]) / float(scale_factor)

def get_score(trees, n):
    if not trees:
        return 0.0
    side = get_side_length(trees)
    return side ** 2 / n

total_score = 0
for n in range(1, 201):
    trees = load_configuration_from_df(n, verify_df)
    total_score += get_score(trees, n)

print(f'Total score: {total_score:.6f}')
print(f'Gap to target: {total_score - 68.922808:.6f}')

Copying repaired baseline to submission...
Submission rows: 20100


Total score: 70.682741
Gap to target: 1.759933


In [9]:
# Compare the two N=2 configurations - they look identical but one has overlap?
print("Comparing N=2 configurations:")
print("\nFailed submission:")
for _, row in failed_df[failed_df['id'].str.startswith('002_')].iterrows():
    print(f"  {row['id']}: x={row['x']}, y={row['y']}, deg={row['deg']}")

print("\nBaseline:")
for _, row in baseline_df[baseline_df['id'].str.startswith('002_')].iterrows():
    print(f"  {row['id']}: x={row['x']}, y={row['y']}, deg={row['deg']}")

Comparing N=2 configurations:

Failed submission:
  002_0: x=s0.154097, y=s-0.038541, deg=s203.629378
  002_1: x=s-0.154097, y=s-0.561459, deg=s23.629378

Baseline:
  002_0: x=s0.154097069621355887, y=s-0.038540742694794648, deg=s203.629377730656841550
  002_1: x=s-0.154097069621372845, y=s-0.561459257305224058, deg=s23.629377730656791812


In [10]:
# The issue is clear: the SA optimizer saved with 6 decimal places, but the baseline has 18+\n# This precision loss caused overlaps!\n\n# Let me check the egortrushin kernel for the grid-based approach\n# This is a fundamentally different approach that builds from scratch\n\nprint(\"Key insight: The SA optimizer failed because of precision loss during save.\")\nprint(\"The baseline uses 18+ decimal places, but the SA output used only 6.\")\nprint(\"\\nNext steps:\")\nprint(\"1. Fix the precision issue in the save code\")\nprint(\"2. Try the egortrushin grid-based translation approach\")\nprint(\"3. Try swap moves from smartmanoj kernel\")